In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate
from credentials import username, password 

# --- Connection Details ---
server = "jarbdatabase.database.windows.net"
database = "Azure_EmployeeDB"
driver_name = "ODBC Driver 18 for SQL Server" 

# 1. Define the clean, non-encoded ODBC connection string
odbc_conn_str = (
    f"DRIVER={{{driver_name}}};" 
    f"SERVER={server};"
    f"DATABASE={database};"
    "Authentication=ActiveDirectoryPassword;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "LoginTimeout=540;"
)

# 2. Create the Engine using the Hybrid URL + connect_args
engine = create_engine(
    # Satisfy SQLAlchemy parser: Use dummy credentials and the driver name in the URL
    f"mssql+pyodbc://a:b@{server}:1433/{database}?driver={driver_name}", 
    connect_args={
        "cnxn_str": odbc_conn_str, 
        "unicode_results": True,
    }
)

# --- Execution ---
print(f"Connection parameters are set for {database}. Waiting for MFA prompt...")

query = "SELECT TOP 10 * FROM [SalesLT].[Customer]"

try:
    df = pd.read_sql(query, engine) 
    print(tabulate(df, headers = 'keys', tablefmt = 'psql'))
    print("\n🎉 Connection Successful! MFA completed and data retrieved.")
except Exception as e:
    print(f"Connection Failed: {e}")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate

# connection details
server = "jarbdatabase.database.windows.net,1433"
database = "Azure_EmployeeDB"
username = "johan.robelto.bayona@johanrobeltobayonaoutlook.onmicrosoft.com"

# IMPORTANT: use the proper ODBC driver installed on your machine
driver = "ODBC Driver 18 for SQL Server"

# create SQLAlchemy engine string with Active Directory Interactive auth
conn_str = (
    f"mssql+pyodbc://{username}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
    f"&Authentication=ActiveDirectoryInteractive"
    f"&Encrypt=yes"
    f"&TrustServerCertificate=no"
)

# create engine
engine = create_engine(conn_str)

# now this is fully supported by pandas
query = "SELECT TOP 10 * FROM [SalesLT].[Customer]"
df = pd.read_sql(query, engine)
df_sql = df[df["CustomerID"] == 1]
print(tabulate(df_sql, headers = 'keys', tablefmt = 'psql'))

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from tabulate import tabulate

# connection details
server = "jarbdatabase.database.windows.net,1433"
database = "Azure_EmployeeDB"
username = "johan.robelto.bayona@johanrobeltobayonaoutlook.onmicrosoft.com"

# IMPORTANT: use the proper ODBC driver installed on your machine
driver = "ODBC Driver 18 for SQL Server"

# create SQLAlchemy engine string with Active Directory Interactive auth
conn_str = (
    f"mssql+pyodbc://{username}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
    f"&Authentication=ActiveDirectoryInteractive"
    f"&Encrypt=yes"
    f"&TrustServerCertificate=no"
)

# create engine
engine = create_engine(conn_str)

# now this is fully supported by pandas
query = "SELECT TOP 10 * FROM [SalesLT].[Customer]"
df = pd.read_sql(query, engine)
df_sql = df[df["CustomerID"] == 1]
print(tabulate(df_sql, headers = 'keys', tablefmt = 'psql'))

In [ ]:
from azure.identity import InteractiveBrowserCredential
import pyodbc
from sqlalchemy import create_engine

server = "jarbdatabase.database.windows.net"
database = "Azure_EmployeeDB"

credential = InteractiveBrowserCredential()
token = credential.get_token("https://database.windows.net/.default")
token_bytes = token.token.encode("utf-16-le")

conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

def connect():
    return pyodbc.connect(conn_str, attrs_before={1256: token_bytes})

engine = create_engine("mssql+pyodbc://", creator=connect)

import pandas as pd
df = pd.read_sql("SELECT TOP 10 * FROM [SalesLT].[Customer]", engine)
print(df.head())

In [3]:
import struct
import pyodbc
from sqlalchemy import create_engine
import pandas as pd
from azure.identity import AzureCliCredential

# Parameters
server = "jarbdatabase.database.windows.net"
database = "Azure_EmployeeDB"

# Acquire token
credential = AzureCliCredential()
token = credential.get_token("https://database.windows.net/.default")

# Convert token to ODBC struct
token_bytes = token.token.encode("utf-8")
exptoken = b"".join(bytes([b]) + b"\0" for b in token_bytes)
token_struct = struct.pack("=i", len(exptoken)) + exptoken

# Build connection string
conn_str = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server};Database={database};TrustServerCertificate=Yes;"

# Create SQLAlchemy engine using pyodbc + token
engine = create_engine(
    f"mssql+pyodbc:///?odbc_connect={conn_str}",
    connect_args={"attrs_before": {1256: token_struct}}
)

# Example: load a table into pandas
df = pd.read_sql("SELECT TOP 10 * FROM [SalesLT].[Customer];", engine)
print(df)


   CustomerID  NameStyle Title  FirstName MiddleName    LastName Suffix  \
0           1      False   Mr.    Orlando         N.         Gee   None   
1           2      False   Mr.      Keith       None      Harris   None   
2           3      False   Ms.      Donna         F.    Carreras   None   
3           4      False   Ms.      Janet         M.       Gates   None   
4           5      False   Mr.       Lucy       None  Harrington   None   
5           6      False   Ms.   Rosmarie         J.     Carroll   None   
6           7      False   Mr.    Dominic         P.        Gash   None   
7          10      False   Ms.   Kathleen         M.       Garza   None   
8          11      False   Ms.  Katherine       None     Harding   None   
9          12      False   Mr.     Johnny         A.      Caprio    Jr.   

                  CompanyName               SalesPerson  \
0                A Bike Store   adventure-works\pamela0   
1          Progressive Sports    adventure-works\david8 